In [1]:
import requests

# Fetch public IP
response = requests.get("http://checkip.amazonaws.com")
if response.status_code == 200:
    public_ip = response.text.strip()
    print(f"Public IP: {public_ip}")
else:
    print(f"Failed to fetch public IP. Status code: {response.status_code}")

# The public IP should be in the whitelist of the AWS server, contact Sabah Hussain for access 

Public IP: 27.18.230.250


In [2]:
from PointSpread import get_quote_data, PointSpreadDisplay

In [5]:
dataInput = get_quote_data("2025-01-08", "2025-01-09","XAU/USD")

Identified partitions: ['p_jan_2025']


2025-01-30 18:01:11,977| ERROR   | Could not open connection to gateway


ERROR fetching data for XAU/USD from 2025-01-08 to 2025-01-09: Could not establish session to SSH gateway


In [4]:
# dataInput = get_quote_data("2025-01-08", "2025-01-09","XAU/USD")
result = PointSpreadDisplay(
            df_input=dataInput,
            trade_vol=300,
            date_range=("2025-01-08", "2025-01-09"),
            maker_id="Britannia",
            top_of_book=False,
            symbol="XAU/USD"
        )


TypeError: 'NoneType' object is not subscriptable